# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [4]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

%run Scripts/calc.py

C:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Antera.py:309: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


LCIA saved at ./Outputs/Antera_original_outputs


In [1]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

input_path = './Data/Antera_inputs_v3.xlsx'
input_sheet = 'Antera_original'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_Antera.xlsx'

iterations = 100
chunks = 'auto'

In [22]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

input_path = './Data/Iris_inputs_v2.xlsx'
input_sheet = 'Iris_AT'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_Iris.xlsx'

iterations = 50000
chunks = 100  #'auto'

In [2]:
# from Tools_Iris import *
# from Model_Iris import *

from Tools_Antera import *
from Model_Antera import *

# from Tools_Antera_upd import *
# from Model_Antera_upd import *

# LCI

The aircraft's life cycle was divided into four parts:
1. Development and Engineering
2. Manufacturing and Assembly
3. Operation
4. End-of-Life

First, we execute function to read inputs from designated `input_path` and transform it into Parameters and ParameterSet object `p`.

In [3]:
inputs = read_inputs(input_path, input_sheet)

In [4]:
p = ParameterSet(inputs, iterations, chunks)

Samples are stored in a xr.Dataset object accessible throught the `p.data` ParameterSet attribute.

Then, we run the function that calculates the functional unit for the study.

In [5]:
p = func_unit(p)

In [6]:
p.data["ha_fleet"].mean().compute().data

array(5.85748173e+09)

In [7]:
p.data["ha_life"].mean().compute()

<xarray.DataArray 'ha_life' ()>
array(713978.75733333)

<br></br>
Executing the UP reading and writing functions.

In [8]:
UP_dataframe = read_unit_processes(database_path)
UP = unit_process_dataset(UP_dataframe)

Building the empty LCI to start the calculation.

In [9]:
inventory = LCI(name=input_sheet, iterations=iterations, UP=UP, parameters=p)
inventory.run()

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 100)
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'm3' 'kg'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Carbon dioxide,...
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Air' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'lower stratosphe...
  * i               (i) int32 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables: (12/16)
    Office          (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Infrastructure  (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Capital         (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Materials       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Factory         (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Logistics       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    ...              ...
    Fuel            (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Recycling       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Incineration    (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Landfill        (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Prototypes      (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Certification   (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
Attributes:
    Name:     Antera_original

The LCI data can be accessed by the `LCI.data` attribute.

# LCIA

Extracting the characterization factors from the excel file located in `database_path`.

Creating an instance of the characterization factors class:

In [10]:
MP_data, EP_data = read_CF(database_path)
CF = CharactFactors(MP_data, EP_data, UP.Substances)

Adjusting the NLT category CFs.

In [11]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

The impacts CFs can be accessed by the `CharactFactors.Midpoint.dataset` or `CharactFactors.Endpoint.dataset` attributes.

In [13]:
display(CF.MP.dataset)
display(CF.EP.dataset)

<xarray.Dataset>
Dimensions:         (Substances: 1835)
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Carbon dioxide,...
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Air' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'lower stratosphe...
Data variables: (12/18)
    FE              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MRD             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    POF             (Substances) float64 0.0 0.0 0.0 0.0 ... 1.0 0.0456 0.0 0.0
    MET             (Substances) float64 0.0 0.0 0.000283 ... 0.0 0.0 0.0
    TA              (Substances) float64 0.0 0.0 0.0 0.0 ... 0.56 0.0 0.0 0.0
    ULO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...              ...
    ALO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    HT              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    IR              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 1.0 0.0
    CC              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 1.0

<xarray.Dataset>
Dimensions:     (Categories: 18)
Coordinates:
  * Categories  (Categories) object 'CC' 'OD' 'TA' 'FE' ... 'WD' 'MRD' 'FD'
Data variables:
    HH          (Categories) float64 1.4e-06 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ED          (Categories) float64 7.93e-09 0.0 5.8e-09 ... 0.0 0.0 0.0
    RA          (Categories) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0715 0.165

<br></br>
**Executing the Impact Assessment.**

In [12]:
aeco = LCIA.build(inventory, CF)

## EOL test calculations

In [15]:
E_sort_constant = 0.4645 / 3.6  # kWh/kg of material, on average
inventory.p["E_sort"] = E_sort_constant * inventory.p['OEW']
LCI_sort = inventory.electricity(inventory.p["E_sort"], "BR")

In [18]:
import xarray as xr
import dask.array as da

In [19]:
materials = ['Al','steel','Ti','inconel','GFRP','CFRP']
scenarios = ['ldf', 'incin','recycl']
#chunks = self.data.chunks['i'][0]
iterations = inventory.data.i.size
UP_eol = inventory.UP.rename_vars({'Landfill':'ldf','Incineration':'incin','Aluminium':'Al',
                                'Titanium':'Ti', 'Inconel':'inconel','Steel':'steel'})
eol = xr.Dataset({scenario: (['Substances','i'],da.empty((1835,iterations), chunks="auto"))
                        for scenario in scenarios}, coords=inventory.data.coords)
        #chunks=(1835,chunks)


In [20]:
for scenario in scenarios:
    for material in materials:
        inventory.p[scenario+"_"+material] = inventory.p["p_"+scenario+"_"+material]*inventory.p[material]
        if scenario == 'recycl':
            eol[scenario] += UP_eol[material] * inventory.p[scenario + "_" + material]
        else:
            eol[scenario] += UP_eol[scenario] * inventory.p[scenario + "_" + material]

In [27]:
test_rec = (LCI_sort - eol['recycl']) / inventory.p["ha_life"]

In [29]:
inventory.data["Materials"].compute()

<xarray.DataArray 'Materials' (Substances: 1835, i: 10000)>
array([[-1.30779063e-13, -1.39008974e-13, -1.43830028e-13, ...,
        -1.34245981e-13, -1.28204776e-13, -1.60505744e-13],
       [-5.68601646e-14, -6.04383543e-14, -6.25344462e-14, ...,
        -5.83675135e-14, -5.57409085e-14, -6.97847374e-14],
       [-4.16704004e-14, -4.46809643e-14, -4.64155383e-14, ...,
        -4.27463353e-14, -4.10481585e-14, -5.12377560e-14],
       ...,
       [-1.61433091e-14, -1.75599282e-14, -1.83999258e-14, ...,
        -1.65723279e-14, -1.60025259e-14, -1.99533944e-14],
       [-5.24163062e-17, -5.70168802e-17, -5.97446907e-17, ...,
        -5.38090700e-17, -5.19596787e-17, -6.47874507e-17],
       [-2.13509974e-11, -2.32249723e-11, -2.43361050e-11, ...,
        -2.19183188e-11, -2.11649970e-11, -2.63901978e-11]])
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'm3' 'kg'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Carbon dioxide,...
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Air' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'lower stratosphe...
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999

In [62]:
LCI_sort.sel(Substances=('Carbon dioxide, fossil', 'Air', 'unspecified')).compute()

<xarray.DataArray (i: 10000)>
array([29.89600993, 31.85322844, 30.09010727, ..., 30.48577777,
       30.18243883, 30.18763832])
Coordinates:
    Substances  object ('Carbon dioxide, fossil', 'Air', 'unspecified')
    Units       object 'kg'
  * i           (i) int32 0 1 2 3 4 5 6 7 ... 9993 9994 9995 9996 9997 9998 9999

In [133]:
lcia_rec = test_rec * CF.MP.array
lcia_rec = lcia_rec.mean('i')
lcia_inc = test_inc * CF.MP.array
lcia_inc = lcia_inc.mean('i')
lcia_ldf = test_ldf * CF.MP.array
lcia_ldf = lcia_ldf.mean('i')
lcia_mat = inventory.data["Materials"] * CF.MP.array
lcia_mat = lcia_mat.mean('i')

In [134]:
lcia_rec_FET = lcia_rec.sel(variable='FET').compute()
lcia_inc_FET = lcia_inc.sel(variable='FET').compute()
lcia_ldf_FET = lcia_ldf.sel(variable='FET').compute()
lcia_mat_FET = lcia_mat.sel(variable='FET').compute()

In [136]:
lcia_rec_FET.sortby(lcia_rec_FET, ascending=False)

<xarray.DataArray (Substances: 1835)>
array([ 0.        ,  0.        ,  0.        , ..., -0.00365486,
       -0.00623476, -0.05052308])
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Carbon dioxide, fossil' ... 'Dioxins...
  - compartment     (Substances) object 'Air' 'Air' 'Air' ... 'Soil' 'Water'
  - subcompartment  (Substances) object 'lower stratosphere + upper troposphe...
    Units           (Substances) object 'kg' 'm3' 'kg' 'kg' ... 'kg' 'kg' 'kg'
    variable        <U3 'FET'

In [135]:
lcia_inc_FET.sortby(lcia_inc_FET, ascending=False)

<xarray.DataArray (Substances: 1835)>
array([0.0290633 , 0.00358653, 0.00210245, ..., 0.        , 0.        ,
       0.        ])
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Dioxins, measured as 2,3,7,8-tetrach...
  - compartment     (Substances) object 'Water' 'Soil' 'Air' ... 'Air' 'Water'
  - subcompartment  (Substances) object 'unspecified' ... 'surface water'
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kg' 'kg'
    variable        <U3 'FET'

In [137]:
lcia_ldf_FET.sortby(lcia_ldf_FET, ascending=False)

<xarray.DataArray (Substances: 1835)>
array([0.05052308, 0.00623476, 0.00365486, ..., 0.        , 0.        ,
       0.        ])
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Dioxins, measured as 2,3,7,8-tetrach...
  - compartment     (Substances) object 'Water' 'Soil' 'Air' ... 'Air' 'Water'
  - subcompartment  (Substances) object 'unspecified' ... 'surface water'
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kg' 'kg'
    variable        <U3 'FET'

In [93]:
lcia_mat.sel(variable='FET').mean().compute()

<xarray.DataArray ()>
array(3.79856235e-07)
Coordinates:
    variable  <U3 'FET'

In [78]:
test_rec.sortby('Substances').compute()

<xarray.DataArray (Substances: 1835, i: 10000)>
array([[-3.41511354e-06, -3.75675405e-06, -3.93009528e-06, ...,
        -3.32054642e-06, -3.54407889e-06, -3.98171864e-06],
       [-3.41511365e-06, -3.75675416e-06, -3.93009539e-06, ...,
        -3.32054652e-06, -3.54407900e-06, -3.98171875e-06],
       [-3.41511351e-06, -3.75675402e-06, -3.93009525e-06, ...,
        -3.32054639e-06, -3.54407886e-06, -3.98171861e-06],
       ...,
       [-3.41511368e-06, -3.75675420e-06, -3.93009542e-06, ...,
        -3.32054656e-06, -3.54407903e-06, -3.98171879e-06],
       [-7.53474743e-07, -9.14884088e-07, -9.79235604e-07, ...,
        -5.96068386e-07, -9.34451910e-07, -7.19491551e-07],
       [-3.27582036e-05, -3.88830664e-05, -4.22493102e-05, ...,
        -3.35563354e-05, -3.43655127e-05, -4.14855095e-05]])
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'venting of nitr...
  - compartment     (Substances) object 'Air' 'Water' ... 'Social' 'Social'
  - subcompartment  (Substances) object 'urban air close to ground' ... nan
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kg' 'kg'
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999

In [79]:
inventory.data["Materials"].sortby('Substances').compute()

<xarray.DataArray 'Materials' (Substances: 1835, i: 10000)>
array([[-5.68601646e-14, -6.04383543e-14, -6.25344462e-14, ...,
        -5.83675135e-14, -5.57409085e-14, -6.97847374e-14],
       [-1.30779063e-13, -1.39008974e-13, -1.43830028e-13, ...,
        -1.34245981e-13, -1.28204776e-13, -1.60505744e-13],
       [-4.16704004e-14, -4.46809643e-14, -4.64155383e-14, ...,
        -4.27463353e-14, -4.10481585e-14, -5.12377560e-14],
       ...,
       [-6.23889229e-14, -6.38432004e-14, -6.47070867e-14, ...,
        -6.42946375e-14, -6.00534296e-14, -7.59786397e-14],
       [-6.23713954e-05, -6.79101733e-05, -7.11775419e-05, ...,
        -6.39710037e-05, -6.18609628e-05, -7.70526278e-05],
       [ 7.55886627e-05,  8.69598076e-05,  9.31460540e-05, ...,
         7.75997624e-05,  7.75451976e-05,  9.48909704e-05]])
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kg' 'kg'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'venting of nitr...
  - compartment     (Substances) object 'Air' 'Water' ... 'Social' 'Social'
  - subcompartment  (Substances) object 'urban air close to ground' ... nan
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999

In [63]:
eol['recycl'].sel(Substances=('Carbon dioxide, fossil', 'Air', 'unspecified')).compute()

<xarray.DataArray 'recycl' (i: 10000)>
array([171.40564264, 180.46765197, 184.87343945, ..., 175.84723061,
       167.85138382, 180.13221589])
Coordinates:
    Substances  object ('Carbon dioxide, fossil', 'Air', 'unspecified')
    Units       object 'kg'
  * i           (i) int32 0 1 2 3 4 5 6 7 ... 9993 9994 9995 9996 9997 9998 9999

In [68]:
test_inc = eol["incin"] / inventory.p["ha_life"]
test_ldf = eol["ldf"] / inventory.p["ha_life"]

In [69]:
aeco.EP

<xarray.Dataset>
Dimensions:         (AOP: 3, i: 10000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
  * AOP             (AOP) <U2 'HH' 'ED' 'RA'
Data variables: (12/16)
    Office          (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Infrastructure  (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Capital         (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Materials       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Factory         (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Logistics       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    ...              ...
    Fuel            (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Recycling       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Incineration    (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Landfill        (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Prototypes      (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Certification   (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
Attributes:
    Name:     Antera_spray

In [34]:
recycl = aeco.get_cat("HH")["Recycling"].compute()
mat = aeco.get_cat("HH")["Materials"].compute()

In [41]:
mat.mean()

<xarray.DataArray 'Materials' ()>
array(3.23162699e-08)
Coordinates:
    AOP      <U2 'HH'

In [40]:
recycl.mean()

<xarray.DataArray 'Recycling' ()>
array(-0.0011687)
Coordinates:
    AOP      <U2 'HH'

## Midpoint Results

Raw midpoint results can be accessed by the `LCIA.MP` attribute. Mean midpoint results are calculated by the `LCIA.mean` method, with pathway set to "MP".

In [41]:
display(aeco.MP)

<xarray.Dataset>
Dimensions:         (Categories: 18, i: 50000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 ... 49995 49996 49997 49998 49999
  * Categories      (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'FD' 'WD' 'CC'
Data variables: (12/16)
    Office          (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Infrastructure  (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Capital         (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Materials       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Factory         (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Logistics       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    ...              ...
    Fuel            (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Recycling       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Incineration    (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Landfill        (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Prototypes      (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Certification   (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>

In [14]:
aeco.mean("MP", by="sum").compute()

<xarray.DataArray 'stack-9786978069e01f90b7ee8b78d22b2a0b' (Categories: 18)>
array([3.47673293e-02, 1.45367512e+06, 9.92854587e-02, 2.27864769e+02,
       3.69835419e-02, 1.29439522e-01, 5.08547577e-02, 1.79333148e+02,
       2.06480898e+02, 5.07248435e-03, 5.20408290e-04, 1.08528287e+00,
       1.63128766e+01, 3.55717401e+05, 8.96515516e-02, 1.52185185e+00,
       9.65473181e-01, 2.28389690e+03])
Coordinates:
  * Categories  (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'IR' 'FD' 'WD' 'CC'

## Endpoint Results

Raw endpoint results can be accessed by the `LCIA.EP` attribute. Mean endpoint results are calculated by the `LCIA.mean` method, with pathway set to "EP".

In [21]:
def cc_flight(lca):
    return lca.get_cat("CC")["Flight"].mean("i").compute().data

In [22]:
cc_flight(aeco)

array(0.64443975)

In [18]:
display(aeco.EP)

<xarray.Dataset>
Dimensions:         (AOP: 3, i: 50000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 ... 49995 49996 49997 49998 49999
  * AOP             (AOP) <U2 'HH' 'ED' 'RA'
Data variables: (12/16)
    Office          (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Infrastructure  (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Capital         (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Materials       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Factory         (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Logistics       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    ...              ...
    Fuel            (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Recycling       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Incineration    (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Landfill        (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Prototypes      (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Certification   (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
Attributes:
    Name:     Antera_spray_mogas

In [35]:
aeco.mean("EP", by="sum").compute()

<xarray.DataArray 'stack-b16e029c735df791a3315e7db32c4566' (AOP: 3)>
array([2.24874163e-01, 6.96987565e-05, 1.28715973e-01])
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

## Contribution to Variance (CTV)

Calculate CTV results for all inputs with midpoint and endpoint results. CTV results can be acessed by the `LCIA.CTV` attribute.

In [13]:
aeco.build_CTV(parameterset=p)

<xarray.Dataset>
Dimensions:     (Parameters: 101)
Coordinates:
  * Parameters  (Parameters) <U18 'takt_US' 'takt_BR' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MRD         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    POF         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MET         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    TA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ULO         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...          ...
    FD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    WD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    CC          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    HH          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ED          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    RA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>

In [16]:
aeco.CTV["CC"].sortby(aeco.CTV["CC"], ascending=False)#.compute()

<xarray.DataArray 'CC' (Parameters: 101)>
dask.array<getitem, shape=(101,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * Parameters  (Parameters) <U18 'pesticide_use' ... 't_cruise_img'

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [14]:
aeco.save(output_path, LCI=False)

LCIA saved at ./Outputs/Antera_updated_outputs


In [22]:
aeco.save(('./Local/'+ input_sheet + '_outputs with LCI'), LCI=True)

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Antera.py:309: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


LCIA saved at ./Local/Antera_spray_outputs with LCI


In [22]:
#aeco.to_excel(output_path)